# Time-integrated point source search analysis example

In [1]:
import os
os.getcwd()

'/data/user/liruohan/prepare_Analysis-BSM-DMannihilation_extragalacticSources/analyses/dm_stacking'

## Software setup

In [2]:
import sys

# Add skyllh and i3skyllh projects to the PYTHONPATH
sys.path.insert(0, '/data/user/liruohan/software/skyllh')
sys.path.insert(0, '/data/user/liruohan/software/i3skyllh')
#sys.path.insert(0, '/home/liruohan/.local/lib/python3.7/site-packages')
#sys.path.insert(0, '/home/cbellenghi/.pyenv/versions/3.8.1/lib/python3.8/site-packages')

# Add missing python packages from cvmfs
#sys.path.insert(0, '/cvmfs/icecube.opensciencegrid.org/py3-v4.1.1/RHEL_7_x86_64/lib/python3.7/site-packages')
extra_path = "/cvmfs/icecube.opensciencegrid.org/users/tkontrimas/software/pip/python3.11/site-packages" # whatever individual directory it is
if extra_path not in sys.path:
    sys.path.append(extra_path)

## Create `datasets` object

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

from skyllh.core.config import Config
from skyllh.core.random import RandomStateService
from skyllh.core.timing import TimeLord
from skyllh.core.source_model import PointLikeSource

# Pre-defined datasets
from i3skyllh.datasets import data_samples

#Pre-defined analysis
# from i3skyllh.analyses.trad_single_ps import analysis as trad_single_analysis
# from i3skyllh.analyses.trad_stacked_ps import analysis as trad_stacking_analysis
# from i3skyllh.analyses.trad_single_ps import analysis_dm as trad_single_analysis_dm
from i3skyllh.analyses.trad_stacked_ps import analysis_dm as trad_stacking_analysis_dm

# import matplotlib as mpl

# mpl.rcParams['figure.figsize'] = (6,4)
# mpl.rcParams['figure.dpi'] = 300

In [4]:
# Update global settings to use 2 cpu cores
#Config['multiproc']['ncpu'] = 2
cfg=Config()
cfg.set_ncpu(2)

from skyllh.core.datafields import (
    DataFieldStages as DFS,
)
# use no MC
# cfg['datafields']['astro'] = DFS.ANALYSIS_MC
# cfg['datafields']['conv'] = DFS.ANALYSIS_MC

In [5]:
# Specify base path for datasets
data_base_path = '/data/ana/analyses'

# Load specific dataset collection
dsc = data_samples['OscNext_v002p04'].create_dataset_collection(
            base_path=data_base_path,cfg=cfg) #or NorthernTracks_v005p01

# Pick a dataset or create a list of datasets from the dataset collection
datasets = dsc.get_datasets('IC86_2011_2021')

In [6]:
from skyllh.core.minimizers.iminuit import IMinuitMinimizerImpl

In [7]:
# Define some common analysis parameters
refplflux_gamma = 0.5
minimizer_impl = IMinuitMinimizerImpl(cfg=cfg)
#minimizer_impl = None
optimize_delta_angle = 10.0
rss_seed = 1
rss = RandomStateService(rss_seed)

In [8]:
ra_arr=np.array([224.755625  , 154.802375  ,   2.02341667, 278.76416667,
        280.5375    , 351.8415    , 299.9994    , 205.2964    ,
        120.68079167, 232.82541667, 104.90875   , 141.321     ,
        234.15958333, 218.71854167, 210.07675   , 141.162     ,
        169.74041667, 141.30375   , 352.6552    , 198.8155    ,
        126.292625  , 118.85541667, 286.35791667,  90.54042   ,
        174.80791667, 174.80791667, 184.47916667, 233.96791667,
        239.79      , 208.26416667, 184.02954167, 219.09208333,
         68.29625   , 195.9975    , 166.1138    ,  73.01958   ,
         34.11292   ,   1.58125   , 139.60833333, 226.005     ,
        171.40083333, 144.55108333, 253.4676    , 196.587375  ,
        323.9766    , 121.41108333, 197.2733    , 259.81041667,
        194.9531    ,  88.72333333,  77.68958333,  70.94537   ,
        115.63666667, 321.164     , 224.27833333, 345.815     ,
         49.95067   ,  37.06079   ,  38.575     , 214.49791667,
        184.61041667, 103.05083333,  12.19642   ,  42.67417   ,
        321.9373    , 304.6606    ,  93.90129   , 307.1462    ,
          6.38092   , 176.31666667, 166.69791667, 186.4455    ,
        183.2627    , 155.8775    , 205.53458333, 180.79      ,
        182.63583333, 182.3745    , 198.365     , 186.45333333,
        184.7397    ])
dec_arr=np.array([61.23152778, 63.96741667, 14.83983333, 32.69638889, 79.77138889,
        15.41036111, 65.14851   , 30.37811   , 31.06758333,  7.45777778,
        54.19666667, 69.488     , 54.55916667, 48.66186111,  5.04505556,
        56.12969444, 58.05666667, 52.28638889, 71.37911   , 44.4075    ,
        37.98880556, 39.18611111, 42.46111111, 28.47139   , 33.93083333,
        33.93083333, 58.65972222, 57.9025    , 35.02972222, 69.30833333,
        50.825     , 58.79416667,  5.35444444, 53.79172   , 38.20883   ,
        49.54583   , 51.42375   , 20.20277778, 16.30527778, 10.43782   ,
        54.3825    ,  7.72777778, 39.76017   , 53.30641667, 47.47453   ,
        26.16816667, 11.63414   , 48.98027778, 27.9807    , 46.43944444,
        16.49888889, 28.97194   , 49.80972222, 50.97329   , 49.669     ,
         8.87388889, 41.5117    , 31.31094   , 32.48333   , 25.13666667,
        29.81277778, 74.42694444, 31.95697   , 54.70419   , 56.94436   ,
        40.68344   , 71.03748   , 25.73361   , 68.36147   , 79.68138889,
        72.56861111, 12.66203   ,  7.03881   , 19.865     , 35.65416667,
        44.53138889, 39.40583333, 43.685     , 36.59358   , 33.54694444,
        47.30397   ])
weight_arr=np.array([0.00040521, 0.00040846, 0.00040989, 0.00041909, 0.00044537,
       0.00045612, 0.00046153, 0.00046779, 0.00047393, 0.00048039,
       0.00048195, 0.00048958, 0.00049103, 0.00049265, 0.00049313,
       0.00051222, 0.00052136, 0.00052444, 0.0005435 , 0.00055509,
       0.0006125 , 0.00065439, 0.00065587, 0.00068172, 0.00069196,
       0.00069196, 0.00069241, 0.00073913, 0.00075588, 0.00076813,
       0.0007818 , 0.00079068, 0.00079232, 0.00081294, 0.00082382,
       0.0008812 , 0.00089339, 0.00092314, 0.00094126, 0.0009752 ,
       0.00097793, 0.00098586, 0.00114846, 0.00116211, 0.00116522,
       0.00117926, 0.00118397, 0.00126644, 0.00138562, 0.00154348,
       0.00155108, 0.00156935, 0.0016089 , 0.00184623, 0.00189567,
       0.00195989, 0.00238219, 0.00264506, 0.00281039, 0.00281405,
       0.00285993, 0.00299   , 0.00323358, 0.00327643, 0.00355436,
       0.00355478, 0.00404473, 0.00435971, 0.00511612, 0.00598944,
       0.00884075, 0.01042823, 0.01501309, 0.02927102, 0.03276768,
       0.04740906, 0.06161564, 0.08164006, 0.08736569, 0.17287162,
       0.34521955])

# most_significant_src={'NGC4258':[184.739700,47.303970,1],'NGC4395':[186.453333,33.546944,1],'NGC5033':[198.365000,36.593580,1],
#                   'NGC4138':[182.374500,43.685000,1],'NGC4151':[182.635833,39.4058331,1],'NGC4051':[180.790000,44.531389,1]}

In [9]:
#Define initial source object
sources = [PointLikeSource(ra=src_ra, dec=src_dec, weight=src_weight)
           for (src_ra, src_dec, src_weight) in zip(np.deg2rad(ra_arr), np.deg2rad(dec_arr), weight_arr)]

In [10]:
# # #source
# single_src=most_significant_src['NGC4258']
#source = PointLikeSource(ra=np.deg2rad(40.75),dec=np.deg2rad(-0.05))

#source_fluxes = DMFlux(channel='W',dm_mass=1000)

In [11]:
# mass_test_list = (200,300,400,500,600,700,800,1000,1200,1500,2000,3000)
# for mass in tqdm(mass_test_list):
#     ana_trad = trad_single_analysis.create_analysis(
#     cfg=cfg,
#     datasets=datasets,
#     source=source,
#     channel='\[Nu]\[Mu]',
#     mass=mass,
#     ns_seed=100.0,
#     compress_data=True,
#     optimize_delta_angle_deg=10,
#     #energy_range=(1e4,2e4),
#     minimizer_impl=minimizer_impl)
    
#     events_list = [ data.mc for data in ana_trad.data_list ]
#     ana_trad.initialize_trial(events_list)
    
#     ns_grid=np.linspace(0,200,201)
#     plt.plot(ns_grid, [ana_trad.llhratio.evaluate([ns, 2.5])[0] for ns in ns_grid],
#              label='mass={}'.format(str(mass)))
#     plt.xlabel(r'$(E[GeV]$')
#     plt.ylabel(r'$-LLHratio$')

In [12]:
# #%%scalene
# ana_trad_powerlaw = trad_single_analysis_powerlaw.create_analysis(
#     cfg=cfg,
#     datasets=datasets,
#     source=source,
# #    bkg_event_rate_field_names=['astro', 'conv'],
#     compress_data=True,
#     optimize_delta_angle_deg=10,
# #    energy_range=(1e4,2e5),
#     ns_seed=100,
#     ns_max=1e4,
#     minimizer_impl=minimizer_impl
# )

In [13]:
# #%%scalene
# ana_single_dm = trad_single_analysis_spline.create_analysis(
#     cfg=cfg,
#     datasets=datasets,
#     source=source,
#     channel='WW',
#     mass=1000,
# #    bkg_event_rate_field_names=['astro', 'conv'],
#     compress_data=True,
#     optimize_delta_angle_deg=10,
#     #energy_range=(1e4,2e4),
#     ns_seed=100,
#     ns_max=1e4,
#     minimizer_impl=minimizer_impl
# )

In [15]:
#%%scalene
ana_stacking_dm = trad_stacking_analysis_dm.create_analysis(
    cfg=cfg,
    datasets=datasets,
    sources=sources,
    channel='WW',
    mass=1000,
#    bkg_event_rate_field_names=['astro', 'conv'],
    compress_data=True,
    optimize_delta_angle_deg=10,
    #energy_range=(1e4,2e4),
    ns_seed=100,
    ns_max=1e4,
    minimizer_impl=minimizer_impl
)

/data/user/liruohan/model_spline/photospline_tables/splinefit_WW_earth_1000_ann.fits


 80%|████████  | 4/5 [00:17<00:00,  1.57it/s]

100%|██████████| 5/5 [00:02<00:00,  2.28it/s]


In [ ]:
break

In [ ]:
(ana_stacking_dm.sig_generator.mu2flux(10.6)) * 9.31257e-06 *1e5

In [ ]:
rss_seed_gen=1
rss_seed_fit=10
rss_gen = RandomStateService(rss_seed_gen)
rss_fit = RandomStateService(rss_seed_fit)

In [ ]:
n_trials=10
for k in range(n_trials):
    # draw random dataset from MC
    (n_sig, n_events_list, events_list) = ana_trad_powerlaw.generate_pseudo_data(rss=rss_gen, mean_n_sig=0)

    # perform analysis on random dataset
#     r_pl = ana_trad_powerlaw.do_trial_with_given_pseudo_data(seed=rss_seed_gen,
#                                                     mean_n_sig=0,
#                                                     n_sig=n_sig,
#                                                     n_events_list=n_events_list,
#                                                     events_list=events_list,
#                                                     minimizer_rss=rss_gen)
    r_sf = ana_trad_dm.do_trial_with_given_pseudo_data(seed=rss_seed_fit,
                                                  mean_n_sig=0,
                                                  n_sig=n_sig,
                                                  n_events_list=n_events_list,
                                                  events_list=events_list,
                                                  minimizer_rss=rss_fit)
    # combine results
#     names = [f'{nn}_sf' for nn in r_sf.dtype.names]
#     vals = [r_sf[nn] for nn in r_sf.dtype.names]
#     r = np.lib.recfunctions.append_fields(r_pl, names, vals, usemask=False, asrecarray=False)
#     results.append(r)

In [ ]:
res = np.lib.recfunctions.stack_arrays(results, asrecarray=False, usemask=False)
print(res)
trials_outfile = os.path.join(args.outdir, args.outfile)
np.save(trials_outfile, res)

### Test signal injection ###

In [ ]:
# mc_pstracks=np.load('/data/ana/analyses/ps_tracks/version-004-p01/IC86_2019_exp.npy')
# mc_pstracks

In [ ]:
background_events = ana_stacking_dm.generate_background_events(rss)
signal_events = ana_stacking_dm.generate_signal_events(rss, 100000)

In [ ]:
print(background_events[1][0])

In [ ]:
print(signal_events[2][0])

In [ ]:
dm_mass=10000
bins=np.linspace(0,dm_mass,100)
bkg_hist=plt.hist(10**(background_events[1][0]['log_energy']),bins=bins,histtype="step",density=True,label='bkg pdf')
sig_hist=plt.hist(10**(signal_events[2][0]['log_energy']),bins=bins,histtype="step",density=True,label='sig pdf')
plt.xlim([0,dm_mass])
plt.legend()

In [ ]:
DM_model = ana_single_dm.shg_mgr.shg_list[0].fluxmodel
DM_model(dm_mass)

In [ ]:
DM_model.mass

In [ ]:
E=np.linspace(0,dm_mass,1000)
plt.plot(E,E**2*DM_model(E),label="spectrum from model")

In [ ]:
plt.figure()
bins = np.linspace(0,dm_mass,100)
plt.hist(bins, weights=DM_model(bins),bins=bins,density=True,alpha=0.2,label="spectrum form model")
bkg_hist=plt.hist(10**(background_events[1][0]['log_energy']),bins=bins,alpha=0.2,density=True,label='bkg pdf')


plt.hist(signal_events[2][0]['true_energy'], bins=bins,histtype="step", label="mc sig true engy", density=True)
plt.hist(ana_single_dm.data_list[0].mc['true_energy'], bins=bins, weights=ana_single_dm.data_list[0].mc['mcweight']*DM_model(ana_single_dm.data_list[0].mc['true_energy']), histtype='step', label='mc sig true engy(re-weighted)', density=True)

plt.hist(10**(signal_events[2][0]['log_energy']), bins=bins,histtype="step", label="mc sig reco engy", density=True)
plt.hist(10**ana_single_dm.data_list[0].mc['log_energy'],bins=bins, weights=ana_single_dm.data_list[0].mc['mcweight']*DM_model(ana_single_dm.data_list[0].mc['true_energy']), histtype='step', label='mc sig reco engy(re-weighted)', density=True)
plt.xlim([0,dm_mass])
plt.legend()

In [ ]:
break

### play with fit and trials ###

In [18]:
# Generate signal trials
rss = RandomStateService(rss_seed)
res_trad_sig = ana_stacking_dm.do_trials(rss, n=1, mean_n_sig=2)
res_trad_sig

  0%|          | 0/1 [00:00<?, ?it/s]

signal_generator.py : ds_idxs [0]
i3/pdfratio gridparams: {'gamma': 0.9}
i3/pdfratio gridparams_hash: -910794440889698088
i3/pdfratio gridparams_hash_log_ratio_spline_dict: {-910794440889698088: <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7fd7db6a1810>, 8308373913968590013: <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7fd7db6b0890>, -723780827370130334: <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7fd7d9fd8ed0>, -1795699540631995886: <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7fd7d9ff6d50>}
i3/pdfratio gridparams: {'gamma': 1.0}
i3/pdfratio gridparams_hash: 8308373913968590013
i3/pdfratio gridparams_hash_log_ratio_spline_dict: {-910794440889698088: <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7fd7db6a1810>, 8308373913968590013: <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7fd7db6b0890>, -723780827370130334: <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7fd7d9fd8ed0>, -179569954063

KeyboardInterrupt: 

In [ ]:
#print(ana_trad.tdm_list[0].get_n_values)

In [17]:
ns_grid=np.linspace(0,100,100)
plt.plot(ns_grid, [ana_stacking_dm.llhratio.evaluate([ns, 1.0])[0] for ns in ns_grid])

In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grads [nan nan]
In llhratio.py log_lambda nan grad

Traceback (most recent call last):
  File "/cvmfs/icecube.opensciencegrid.org/py3-v4.3.0/Ubuntu_22.04_x86_64/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_6693/3277087923.py", line 2, in <module>
    plt.plot(ns_grid, [ana_stacking_dm.llhratio.evaluate([ns, 1.0])[0] for ns in ns_grid])
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_6693/3277087923.py", line 2, in <listcomp>
    plt.plot(ns_grid, [ana_stacking_dm.llhratio.evaluate([ns, 1.0])[0] for ns in ns_grid])
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/user/liruohan/software/skyllh/skyllh/core/llhratio.py", line 1176, in evaluate
    (log_lambda_j, grads_j) = llhratio.evaluate(
                              ^^^^^^^^^^^^^^^^^^
  File "/data/user/liruohan/software/skyllh/skyllh/core/llhratio.py", line 882, in evalua

In [ ]:
break

In [ ]:
import os
import scipy.optimize
import scipy.stats as st
from matplotlib.colors import LinearSegmentedColormap
ts_cmap = LinearSegmentedColormap.from_list('mycmap', ['white', 'red', '#800000'])
import math
import matplotlib as mpl

In [ ]:
x_all = ana_trad.data_list[0].mc
x= x_all[x_all['dec']<0]

In [ ]:
print(ana_trad.data_list[0].mc)

In [ ]:
min(x_all['dec']),min(x['dec'])

In [ ]:
print(np.min(x['log_energy']),np.max(x['log_energy']))

In [ ]:
tot_weight = x['mcweight']*1.*1e-18*(x['true_energy']/1e5)**(-2.)

In [ ]:
len(x['run']),len(np.unique(x['run']))

In [ ]:
run_id = np.array([str(int(i)) for i in x['run']])
ev_id = np.array([str(int(i)) for i in x['time']])
unique_id = [run_id[i] + ev_id[i] for i in range(len(ev_id))]

bins_Ex = np.linspace(2,8, 100)
bins_Ey = np.linspace(2,8, 100)

In [ ]:
def get_quantiles(truth, predictions, bins, weight, quantiles=[0.1, 0.5, 0.9]):
    ret_dict = {}
    for i in quantiles:
        ret_dict[i]=[]
    for j in quantiles:
        for i in range(len(bins_Ex)-1):
            mask = (bins[i]<truth) & (bins[i+1]>truth) & (np.isfinite(predictions))
            wq = weighted_quantile(predictions[mask], j, sample_weight=weight[mask])
            ret_dict[j].append(wq)
    return ret_dict

def weighted_quantile(values, quantiles, sample_weight=None, 
                      values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: numpy.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of
        initial array
    :param old_style: if True, will correct output to be consistent
        with numpy.percentile.
    :return: numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), \
        'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

In [ ]:
def make_energy_2d_plot(prediction, truth, tot_weight, bins_Ex, bins_Ey, savepath):
    quantiles_dnn = get_quantiles(truth, prediction, bins_Ey, tot_weight, quantiles=[0.05, 0.5, 0.95])
    fig = plt.figure(figsize=(8, 7))
    ax = fig.add_subplot(111)

    H = np.histogram2d(truth, prediction, bins = [bins_Ey, bins_Ex], weights=tot_weight)
    H1 = np.histogram(truth, bins = bins_Ex, weights=tot_weight)
    H_plot = H[0]/H1[0][:, np.newaxis]
    save_dict = dict()
    save_dict['bins_true_E'] = bins_Ex
    save_dict['bins_reco_E'] = bins_Ey
    save_dict['bin_vals'] = H_plot
    np.save('reco_energy.npy', save_dict)
    print(np.max(H_plot))
    cbar = ax.pcolormesh(bins_Ey, bins_Ex, H_plot.T, cmap=ts_cmap, norm=mpl.colors.LogNorm(vmin=1e-5, vmax=0.5))
    plt.colorbar(cbar, label=r'$P(E_{reco} | E_{\mu})$')
    ax.plot([1,8], [1,8], linestyle='-', color = 'k')
    ax.plot(bins_Ex[:-1], quantiles_dnn[0.05], color='white',lw=1.5, ls='--')
    ax.plot(bins_Ex[:-1], quantiles_dnn[0.5], color='white', lw=1.5)
    ax.plot(bins_Ex[:-1], quantiles_dnn[0.95], color='white', lw=1.5, ls='--')
    
    bins_Ex_greco = np.linspace(1.,3.5, 50)
    bins_Ey_greco = np.linspace(1.,3.5, 50)
#     energy_greco=np.load('reco_energy_greco.npy',allow_pickle=True)
#     cbar2 = ax.pcolormesh(bins_Ey_greco, bins_Ex_greco, (energy_greco.item()['bin_vals']).T,
#                           cmap=ts_cmap, norm=mpl.colors.LogNorm(vmin=1e-5, vmax=0.5))
#     plt.colorbar(cbar2)
    
    quantile_greco=np.load('quantile_dnn_greco.npy', allow_pickle=True)
    ax.plot(bins_Ex_greco[:-1], quantile_greco.item()[0.05], color='black',lw=1.5, ls='--')
    ax.plot(bins_Ex_greco[:-1], quantile_greco.item()[0.5], color='black', lw=1.5)
    ax.plot(bins_Ex_greco[:-1], quantile_greco.item()[0.95], color='black', lw=1.5, ls='--')
    
    ax.set_ylim(1.,8)
    ax.set_xlim(1.,8)
    ax.set_xlabel(r'$\log_{10}(E_{\mu}/GeV)$')
    ax.set_ylabel(r'$\log_{10}(E_{reco}/GeV)$')
    plt.grid(True)
    fig.savefig(savepath, dpi=300, bbox_inches='tight')
    fig.show()

In [ ]:
energy.item()

In [ ]:
truth = np.log10(x['true_energy'])
prediction = x['log_energy']

make_energy_2d_plot(prediction, truth, tot_weight, bins_Ex, bins_Ey, 'dnn.png')

In [ ]:
events_list = [data.mc for data in ana_trad.data_list]
ana_trad.initialize_trial(events_list)

In [ ]:
rss = RandomStateService(seed=1)

In [ ]:
(log_lambda_max, fitparam_values, status) = ana_trad.llhratio.maximize(rss)

In [ ]:
print(f'log_lambda_max = {log_lambda_max}')
print(f'fitparam_values = {fitparam_values}')
print(f'status = {status}')

In [ ]:
TS = ana_trad.calculate_test_statistic(log_lambda_max, fitparam_values)
print(f'TS = {TS:.3f}')

In [ ]:
(ts, x, status) = ana_kde.unblind(rss=rss)
print(f'TS = {ts:.3f}')
print(f'ns = {x["ns"]:.2f}')
print(f'gamma = {x["gamma"]:.2f}')

In [ ]:
scaling_factor = ana_kde.calculate_fluxmodel_scaling_factors(x['ns'], [x['ns'], x['gamma']])
print(f'Flux scaling factor = {scaling_factor[0]:.3e}')
print(f'{scaling_factor[0]:.3e}'' (E/1000 GeV)^{-'f'{x["gamma"]:.2f}'+'} 1/(GeV s cm^2 sr)')

In [ ]:
(llhratio_value, (grad_ns, grad_gamma)) = ana_kde.llhratio.evaluate([14.62, 2.59])
print(f'llhratio_value = {llhratio_value:.3f}')
print(f'grad_ns = {grad_ns:.3f}')
print(f'grad_gamma = {grad_gamma:.3f}')

In [ ]:
ana_kde.sig_generator.mu2flux(14.62)

In [ ]:
from tqdm import tqdm

In [ ]:
(ns_min, ns_max, ns_step) = (0, 80, 0.5)
(gamma_min, gamma_max, gamma_step) = (1.5, 4.0, 0.1)

ns_edges = np.linspace(ns_min, ns_max, int((ns_max-ns_min)/ns_step)+1)
ns_vals = 0.5*(ns_edges[1:] + ns_edges[:-1])

gamma_edges = np.linspace(gamma_min, gamma_max, int((gamma_max-gamma_min)/gamma_step+1))
gamma_vals = 0.5*(gamma_edges[1:] + gamma_edges[:-1])

delta_ts = np.empty((len(ns_vals), len(gamma_vals)), dtype=np.double)
for (ns_i, ns) in tqdm(enumerate(ns_vals)):
    for (gamma_i, gamma) in enumerate(gamma_vals):

        delta_ts[ns_i, gamma_i] = (
            ana_kde.calculate_test_statistic(llhratio_value, [14.58, 2.17]) -
            ana_kde.calculate_test_statistic(ana_kde.llhratio.evaluate([ns, gamma])[0], [ns, gamma])
        )

# Determine the best fit ns and gamma values from the scan.
index_max = np.argmin(delta_ts)
ns_i_max = int(index_max / len(gamma_vals))
gamma_i_max = index_max % len(gamma_vals)
ns_best = ns_vals[ns_i_max]
gamma_best = gamma_vals[gamma_i_max]

In [ ]:
# Determine the delta lambda value for the 95% quantile assuming a chi-sqaure
# distribution with 2 degrees of freedom (i.e. assuming Wilks theorem).
chi2_68_quantile = scipy.stats.chi2.ppf(0.68, df=2)
chi2_90_quantile = scipy.stats.chi2.ppf(0.90, df=2)
chi2_95_quantile = scipy.stats.chi2.ppf(0.95, df=2)

In [ ]:
from matplotlib.colors import LogNorm
plt.figure(figsize=(8,6))
plt.pcolormesh(gamma_edges, ns_edges, delta_ts, cmap='nipy_spectral')
cbar = plt.colorbar()
cbar.set_label(r'$\Delta$TS')
plt.contour(gamma_vals, ns_vals, delta_ts, [chi2_68_quantile], colors='#FFFFFF')
plt.contour(gamma_vals, ns_vals, delta_ts, [chi2_90_quantile], colors='#AAAAAA')
plt.contour(gamma_vals, ns_vals, delta_ts, [chi2_95_quantile], colors='#444444')
plt.plot(gamma_best, ns_best, marker='x', color='white', ms=10)
plt.xlabel(r'$\gamma$')
plt.ylabel(r'$n_{\mathrm{s}}$')
plt.ylim(ns_min, ns_max)
plt.xlim(gamma_min, gamma_max)

## Some utility functions

In [ ]:
from skyllh.core.utils.analysis import (estimate_sensitivity,
    estimate_discovery_potential,
    create_trial_data_file,
    estimate_sen_dep_from_trials
)

In [ ]:
help(create_trial_data_file)

In [ ]:
help(estimate_sensitivity)

In [ ]:
sensitivity_results=estimate_sensitivity(ana_trad,rss,mu_range=(10,20),pathfilename='/home/liruohan/cluster_run/dm_test_sensitivity_1src_800GeV.npy')

In [ ]:
print(sensitivity_results)

In [ ]:
with open('/home/liruohan/single_source/sensitivity_results.txt', 'a+') as f:
    f.write(source_name +" DM 800GeV:"+str(sensitivity_results)+"\n")
    f.close()

In [ ]:
n_trials = 2500
pathfilename = 'combined_trails.npy'
_ = create_trial_data_file(ana_kde, rss, n_trials, mean_n_sig=0, mean_n_sig_null=0, pathfilename=pathfilename)

In [ ]:
trials = np.load(pathfilename)
print(trials)
print(trials.dtype)

## Pseudo data example

In [ ]:
# Generate background events.
(bg_n_events_list, bg_events_list) = ana_kde.generate_background_events(
    rss=rss,
)

# Generate signal events.
(n_sig, sig_n_events_list, sig_events_list) = ana_kde.generate_signal_events(
    rss=rss,
    mean_n_sig=10
)

In [ ]:
print(sig_events_list[0])

In [ ]:
delta_angle = 3.0

plt.figure(figsize=(8, 8))

# Scatter background and signal events.
plt.scatter(np.rad2deg(bg_events_list[0]['ra']), np.rad2deg(bg_events_list[0]['dec']), label='bg', alpha=0.3, edgecolors='none')
plt.scatter(np.rad2deg(sig_events_list[0]['ra']), np.rad2deg(sig_events_list[0]['dec']), label='sig')
plt.scatter(np.rad2deg(new_source.ra), np.rad2deg(new_source.dec), marker='*', label='source', s=150)

plt.xlabel('ra, deg')
plt.ylabel('dec, deg')
plt.xlim(np.rad2deg(new_source.ra) - delta_angle, np.rad2deg(new_source.ra) + delta_angle)
plt.ylim(np.rad2deg(new_source.dec) - delta_angle, np.rad2deg(new_source.dec) + delta_angle)
plt.legend()